In [1]:
import pandas as pd
df = pd.read_csv(r'/Users/adamchapnik/Desktop/CCP_Resumes/after92bio.csv')
df.dropna(subset = ["experience"], inplace=True)
df = df[["name", "experience"]]
df

,name,experience
0,陈希同,1948年，考入北京大学中文系，同年10月，加入中国民主青年同盟
1,陈希同,1949年12月，加入中国共产党
2,陈希同,曾任北京市西单区街道工作组组员，市公安局内二分局十二派出所副所长，内二分局人事股副股长、文书...
3,陈希同,1953年后，在中共北京市委办公厅工作，任刘仁同志秘书，北京第一机床厂车间党支部副书记
4,陈希同,1963年后，任北京市昌平县委农工部副部长、县委副书记
...,...,...
6344,王乐泉,2012年至2013年中央政法委副书记
6345,王乐泉,2013年11月至2019年03月中国法学会会长
6346,王乐泉,第十四届中央候补委员，十五届、十六届、十七届中央委员，十六届、十七届中央政治局委员
6347,王乐泉,王乐泉任免信息2019年3月20日，中国法学会第八次全国会员代表大会在京闭幕


In [2]:
bio = df["experience"]
name = df["name"]

###SKIP###
## sentence segment bios ##
import numpy as np
import itertools
import spacy

nlp = spacy.load("zh_core_web_sm")

full_df = [] # create list for dfs

for x in range(len(bio)): # count number of long_bios
    
    segs = [] # create empty list for setnences
    doc = nlp(bio[x]) # run long_bio x through model
    
    for sent in doc.sents:
        segs.append(sent.text) # append each segmented sentence to empty list segs

    df = pd.DataFrame() # create empty data frame
    df2 = df.assign(Experience = segs) # create column in df with list segs

    names = [] # create empty list for names

    for _ in itertools.repeat(None, len(df2.index)): # count number sentences
        names.append(name[x]) # repeat name x number times of sentences
    
    df2.insert(0, "Name", names, True) # add column to df for names
    
    full_df.append(df2)
    
full_df = pd.concat(full_df)
full_df.to_csv('~/Desktop/Segmented_Bios.csv', index=False, encoding='utf-8')

In [4]:
full_df = df
# Named Entity Recognition (Spacy) ##
import numpy as np
import itertools
import spacy

nlp = spacy.load("zh_core_web_sm")

matrix = []

for index, row in full_df.iterrows():
    sent = row['experience'] 
    doc = nlp(sent)
    
    label = [(e.label_) for e in doc.ents]  
    col = ['Name', 'Segment']
    row1 = col + label

    ent = [(e.text) for e in doc.ents]
    row2 = row.tolist() + ent

    ext = row1, row2
    
    ext2 = pd.DataFrame(ext)
    
    for x in range(2, len(ext2.columns)):
        if any(i == ext2[x][0] for i in ['ORG', 'DATE', 'LOC', 'EVENT', 'GPE', 'NORP']): # entity types to KEEP
            pass
        else:
            ext2 = ext2.drop([x], axis = 1)
        
    ext3 = ext2.to_numpy().tolist()
    matrix = matrix + ext3
    
df = pd.DataFrame(matrix)
df.to_csv('~/Desktop/NER_Version_4.csv', index=False, encoding='utf-8')

Following the NER, I will now complete relation extraction to split experiences into individual rows/sets.

I will use SpaCy dependency matching for sentences with the structure of a data and one or more experiences.

In [62]:
import spacy
from spacy import displacy

import pandas as pd

nlp = spacy.load("zh_core_web_sm")

df = pd.read_csv(r'/Users/adamchapnik/Desktop/CCP_Resumes/Complete_NER_1.csv', encoding='utf-8')
df

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43
0,labs,Name,Segment,DATE,ORG,TITL,TITL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ner,阿不来提阿不都热西提,1960至1965年新疆工学院机电系电气专业学习,1960至1965年,新疆工学院机电系电气专业学习,学院,学习,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,labs1,Name,Segment,DATE,GPE,TITL,TITL,TITL,TITL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ner1,阿不来提阿不都热西提,1965至1980年新疆维吾尔自治区建筑勘察设计院技术员、工程师、专业组长,1965至1980年,新疆,组长,技术员,工程师,技术员、工程师,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,labs2,Name,Segment,DATE,GPE,TITL,TITL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18069,ner9034,朱小丹,2012年至2015年，广东省委副书记、省长,2012年至2015年,广东省委,副书记,书记,省长,省委副书记,委副书记,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18070,labs9035,Name,Segment,DATE,ORG,GPE,GPE,ORG,TITL,TITL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18071,ner9035,朱小丹,2015年至2016年，广东省委副书记、省长，中国（广东）自由贸易试验区工作领导小组组长,2015年至2016年,广东省委,中国,广东,自由贸易试验区,组长,副书记,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18072,labs9036,Name,Segment,DATE,EVENT,TITL,TITL,TITL,TITL,TITL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
text = df['X2']
text = text[3]
text

'1965至1980年新疆维吾尔自治区建筑勘察设计院技术员、工程师、专业组长'

In [142]:
import re
import numpy as np
# get extracted titles and remove redundancies
for index, col in df.iloc[2:4].iteritems():
    if col[2] == "TITL":
        strings = col[3]
        tests = re.findall(strings, col[3])
        for test in tests:
            if test == col[3]:  
                x = [item for sublist in [col[3]] for item in sublist]
                x = max(x, key=len)
                print([[col[3]]])

['组长']
['技术员']
['工程师']
['技术员、工程师']


In [146]:
import spacy
from spacy import displacy

nlp = spacy.load("zh_core_web_sm")
doc = nlp('广东省委副书记、省长，中国（广东）自由贸易试验区工作领导小组组长')
displacy.serve(doc)


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [144]:
for tok in doc:
    print(tok.text,"-->",tok.dep_,"-->",tok.pos_)

新疆 --> compound:nn --> PROPN
维吾尔 --> compound:nn --> PROPN
自治区 --> compound:nn --> NOUN
建筑 --> compound:nn --> NOUN
勘察 --> compound:nn --> NOUN
设计院 --> compound:nn --> NOUN
技术员 --> conj --> NOUN
、 --> punct --> PUNCT
工程师 --> conj --> NOUN
、 --> punct --> PUNCT
专业 --> compound:nn --> NOUN
组长 --> ROOT --> NOUN


In [57]:
date_entities = [ent for ent in doc.ents if ent.label_ == "DATE"]
for ent in date_entities:
    head = ent.root.head
head

组长

In [47]:
org_entities = [ent for ent in doc.ents if ent.label_ == "LOC"]
print(org_entities)

[]


In [36]:
print(spacy.explain("nmod:assmod"))

None
